In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# other imports/set up
import keras
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.applications.vgg16 import vgg16

In [ ]:
# import the data and specify the paths
sample_submission = pd.read_csv("../input/aptos2019-blindness-detection/sample_submission.csv")
test = "../input/aptos2019-blindness-detection/test_images"
train = "../input/aptos2019-blindness-detection/train_images"
df_test = pd.read_csv("../input/aptos2019-blindness-detection/test.csv")
df_train = pd.read_csv("../input/aptos2019-blindness-detection/train.csv")

In [ ]:
# look at the info of the training data
df_train.info()

In [ ]:
# look at the head of the training data
df_train.head()

In [ ]:
#https://www.kaggle.com/carlolepelaars/efficientnetb5-with-keras-aptos-2019
print("Image IDs and Labels (TRAIN)")
# Add extension to id_code
df_train['id_code'] = df_train['id_code'] + ".png"
print(f"Training images: {df_train.shape[0]}")
display(df_train.head())
print("Image IDs (TEST)")
# Add extension to id_code
df_test['id_code'] = df_test['id_code'] + ".png"
print(f"Testing Images: {df_test.shape[0]}")
display(df_test.head())

In [ ]:
# check columns
for i in df_train:
    print(i)

In [ ]:
# look at train info
df_train.info()

In [ ]:
# look at train examples
df_train.head()

In [ ]:
# image data generators
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   validation_split=0.2,
                                  rotation_range=360,
                                  horizontal_flip=True,
                                  vertical_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# train generator
train_generator = train_datagen.flow_from_dataframe(df_train,
                                                    x_col='id_code', 
                                                    y_col='diagnosis', 
                                                    directory=train, 
                                                    target_size=(244,244), 
                                                    class_mode='raw',
                                                   subset='training')

# validation generator
val_generator = train_datagen.flow_from_dataframe(df_train,
                                                 x_col='id_code',
                                                 y_col='diagnosis',
                                                 directory=train,
                                                 target_size=(244,244),
                                                 class_mode='raw',
                                                 subset='validation')


In [ ]:
# get vgg16 model 
model_vgg16 = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet',input_shape=(244,244,3))

In [ ]:
# vgg16 summary
model_vgg16.summary()

In [ ]:
# make it so the vgg models are not trainable
for layers in model_vgg16.layers:
    layers.trainable = False

In [ ]:
# create a model, add fcn to the vgg model
model = Sequential()
model.add(model_vgg16)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.summary()

In [ ]:
# compile the model
model.compile(loss='sparse_categorical_crossentropy',
             optimizer=keras.optimizers.Adam(),
             metrics=['accuracy'])

In [ ]:
# fit the model
history = model.fit_generator(train_generator,
                             epochs=20,
                             validation_data=val_generator,
                             verbose=1)

In [ ]:
# check the model performance
history.history

In [ ]:
# function to plot the loss
def train_validation_loss_plot():
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(1, len(loss) + 1)
    
    plt.plot(epochs, loss, label='Training Loss')
    plt.plot(epochs, val_loss, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    return plt.show()

In [ ]:
# plot loss
train_validation_loss_plot()

In [ ]:
# function to plot accuracy
def train_validation_accuracy_plot():
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    
    epochs = range(1, len(acc) + 1)
    
    plt.plot(epochs, acc, label='Training Accuracy')
    plt.plot(epochs, val_acc, label='Validation Accuracy')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    return plt.show()

In [ ]:
# plot the accuracy
train_validation_accuracy_plot()